In [1]:
import numpy as np
from IPython.display import display
from ipywidgets import HTML

In [2]:
board = ((0, 0, 0, 0, 0, 0, 0),
         (0, 1, 1, 1, 0, 1, 0),
         (0, 1, 1, 1, 1, 1, 0),
         (0, 1, 1, 1, 1, 1, 0),
         (0, 1, 1, 1, 1, 1, 0),
         (0, 0, 0, 0, 0, 0, 0))

# Floor/Wall/Target.
target = (1, 5)

# Characters.
enemy = (1, 1)
player = [3, 5]

# Objects.
rocks = set([(2, 3)])
water = set([(2, 5)])
keys = set([(4, 2)])
crosses = set([(2, 1), (3, 1), (4, 1)])

In [44]:
def get_board_sizes():
    return len(board), len(board[0])

def is_floor(coord):
    try:
        x, y = coord
        return board[x][y] == 1
    except IndexError:
        return False

In [30]:
def get_content(coord):
    coord = tuple(coord)
    content = [None, None]

    # Floor/Wall/Target.
    if coord == target:
        content[0] = ("target")
    elif is_floor(coord):
        content[0] = ("floor")
    else:
        content[0] = ("wall")

    # Objects.
    if coord in rocks:
        content[1] = ("rock")
    elif coord in water:
        content[1] = ("water")
    elif coord in crosses:
        content[1] = ("cross")
    elif coord in keys:
        content[1] = ("key")

    # Character.
    if list(coord) == player:
        content[1] = ("player")
    elif coord == enemy:
        content[1] = ("enemy")

    return content

In [5]:
def is_valid(coord):
    return is_floor(coord)

def is_rocks_free(coord):
    return is_valid(coord) and not tuple(coord) in rocks

def is_player_free(coord):
    return is_rocks_free(coord) and not tuple(coord) in water

In [6]:
def remove_crosses_from(coord):
    new_crosses = set(crosses)
    sum_initial_cross = sum(coord)
    enemy_is_facing_down = sum(enemy) < sum_initial_cross

    for cross in crosses:
        cross_sum = sum(cross)
        if ((enemy_is_facing_down
             and cross_sum >= sum_initial_cross)
            or
            (not enemy_is_facing_down
                and cross_sum <= sum_initial_cross)):
            new_crosses.remove(cross)

    return new_crosses

In [39]:
def move_player(mov):
    new_player = player
    new_keys = set(keys)

    next_coord = tuple(np.add(new_player, mov))

    if is_player_free(next_coord):
        new_player = list(next_coord)
        # If players picks the key => Remove key.
        if next_coord in new_keys:
            new_keys.remove(next_coord)
        # If players wins => Remove player.
        elif next_coord == target:
            new_player = None

    return new_player, new_keys


def move_rocks(mov):
    new_water = set(water)
    new_crosses = set(crosses)
    new_rocks = set(rocks)

    next_coord = tuple(np.add(player, mov))
    next_rock_space = tuple(np.add(next_coord, mov))

    if (next_coord in new_rocks
            and is_rocks_free(next_rock_space)):
        new_rocks.remove(next_coord)
        # We check if water in new rock position.
        if next_rock_space in new_water:
            new_water.remove(next_rock_space)
        else:
            if next_rock_space in crosses:
                new_crosses = remove_crosses_from(next_rock_space)
            new_rocks.add(next_rock_space)

    return new_rocks, new_water, new_crosses

def move(mov):
    new_rocks, new_water, _ = move_rocks(mov)
    new_player, _ = move_player(mov)
    return new_player, new_rocks, new_water

In [8]:
element_image = {
    "floor": "./sprites/floor.png",
    "wall": "./sprites/wall.png",
    "target": "./sprites/target.png",
    "player": "./sprites/player.png",
    "rock": "./sprites/rock.png",
    "water": "./sprites/water.png",
    "key": "./sprites/key.png",
    "enemy": "./sprites/enemy.png",
    "cross": "./sprites/cross.gif"
}

In [46]:
# Characters.
enemy = (1, 1)
player = [2, 2]

# Objects.
rocks = set([(2, 3)])
water = set([(2, 5)])
keys = set([(4, 2)])
crosses = set([(2, 1), (3, 1), (4, 1)])


def get_html():
    height, width = get_board_sizes()

    new_row = "<tr>"
    end_row = "</tr>"

    html_string = "<style> img.game {width: 50px !important; height: 50px !important;}</style><table>"
    
    for i in range(height):
        html_string += new_row
        for j in range(width):
            content = get_content((i, j))
            if not content[1]:
                drawing = element_image[content[0]]
            else:
                drawing = element_image[content[1]]
            html_string += f'<td><img class="game" src={drawing} alt=""></img></td>'
        html_string += end_row
    html_string += "</table>"
                
    return html_string

HTML(get_html())

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [47]:
from IPython.display import display

# "Restarting" the values, for testing porpuses.

# Characters.
enemy = (1, 1)
player = [2, 2]

# Objects.
rocks = set([(2, 3)])
water = set([(2, 5)])
keys = set([(4, 2)])
crosses = set([(2, 1), (3, 1), (4, 1)])

display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [49]:
# ================================================
# El jugador empuja la piedra, y se mueve
rocks, water, crosses = move_rocks([0, 1])
player, key = move_player([0, 1])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [50]:
# ========================================================================
# El jugador empuja la piedra y se mueve, desaparece el agua, y la piedra
rocks, water, crosses = move_rocks([0, 1])
player, key = move_player([0, 1])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [53]:
# ================================================================
# El jugador se mueve hasta la última posición libre de la fila
player, key = move_player([0, 1])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [54]:
# =======================================================================
# El jugador se mueve hacia arriba, y entra en la meta (desapareciendo)
player, key = move_player([-1, 0])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [15]:
from IPython.display import display

# "Restarting" the values, for testing porpuses.

# Characters.
enemy = (1, 1)
player = [2, 2]

# Objects.
rocks = set([(2, 3)])
water = set([(2, 5)])
keys = set([(4, 2)])
crosses = set([(2, 1), (3, 1), (4, 1)])

display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [16]:
# ==========================================================================================================================
# El jugador empuja la piedra y se mueve, desaparece el aspa, pero no la piedra, y, además, el conjunto de aspas por debajo
# de la primera también desaparece, dando a entender que la piedra protege a quien se ponga en cualquier casilla por 
# debajo de dicha piedra. 
# ==========================================================================================================================
rocks, water, crosses = move_rocks([0, -1])
player, key = move_player([0, -1])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [31]:
from IPython.display import display

# "Restarting" the values, for testing porpuses.

# Characters.
enemy = (1, 1)
player = [2, 2]

# Objects.
rocks = set([(2, 3)])
water = set([(2, 5)])
keys = set([(4, 2)])
crosses = set([(2, 1), (3, 1), (4, 1)])

display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [32]:
# ===============================================================
# Muevo el jugador hacia bajo respecto de su posición inicial

player, key = move_player([1, 0])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [34]:
# =============================================================================================
# Muevo el jugador hacia la izquierda. La idea es que el jugador pase a la posición de la llave, y, a partir de ese momento, la 
# llave ya no vuelva a aparecer con símbolo, en el tablero.

player, key = move_player([0, -1])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [43]:
# ==================================================================================================
# Muevo el jugador hacia la derecha desde la posición anterior. La posición de la llave debería ser substituida por suelo.

player, keys = move_player([0, 1])
display(HTML(get_html()))

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…